In [1]:
import datetime
import sys
import dns.message
import dns.rdataclass
import dns.rdatatype
import dns.query
import time

from dns.rrset import RRset


name="paypal.com"
rootservers = ['198.41.0.4',
'192.228.79.201', 
'192.33.4.12',
'199.7.91.13',
'192.203.230.10',
'192.5.5.241',
'192.112.36.4',
'198.97.190.53',
'192.36.148.17',
'192.58.128.30',
'193.0.14.129',
'199.7.83.42',
'202.12.27.33']

rdtype_dic = {
    'A': 1      ,
    'NS':2      ,
    'DS':43     ,
    'RRSIG': 46 ,
    'DNSKEY':48 ,
}


a_query = dns.message.make_query('paypal.com', 'A', want_dnssec=True)
response1 = dns.query.udp(a_query, '198.41.0.4')
print(response1)
print("*"*50)
b_query = dns.message.make_query('.', 'DNSKEY', want_dnssec=True)
response2 = dns.query.udp(b_query, '198.41.0.4')
print(response2)
# print(print("".join(response2.answer[0].to_text().split("\n")[0].split(" ")[7:])))
print(dns.dnssec.make_ds(".", response2.answer[0][0], "SHA256", origin=None))
key=dns.name.from_text(".")
print(type(key))
print(response1.authority[0])
# print(response1.authority[2])
value = dns.dnssec.validate(response2.answer[0], response2.answer[1], {key:response2.answer[0]})
value = dns.dnssec.validate(response1.authority[1], response1.authority[2], {key:response2.answer[0]})

print(value)

id 5269
opcode QUERY
rcode NOERROR
flags QR RD
edns 0
eflags DO
payload 4096
;QUESTION
paypal.com. IN A
;ANSWER
;AUTHORITY
com. 172800 IN NS e.gtld-servers.net.
com. 172800 IN NS b.gtld-servers.net.
com. 172800 IN NS j.gtld-servers.net.
com. 172800 IN NS m.gtld-servers.net.
com. 172800 IN NS i.gtld-servers.net.
com. 172800 IN NS f.gtld-servers.net.
com. 172800 IN NS a.gtld-servers.net.
com. 172800 IN NS g.gtld-servers.net.
com. 172800 IN NS h.gtld-servers.net.
com. 172800 IN NS l.gtld-servers.net.
com. 172800 IN NS k.gtld-servers.net.
com. 172800 IN NS c.gtld-servers.net.
com. 172800 IN NS d.gtld-servers.net.
com. 86400 IN DS 30909 8 2 e2d3c916f6deeac73294e8268fb5885044a833fc5459588f4a9184cfc41a5766
com. 86400 IN RRSIG DS 8 1 86400 20211005210000 20210922200000 26838 . bF/r3zrgxQLjC3hituZPxmzTURcKJ3nc 8BjFu0eMJwxovRYJBQwLetYcWL0c5HEl GkzGA/wWZJ1XjxRaCpIaw040WbVwX7+T CKR3nrNS8XbKWb/u5NRZERafRKxw+BbU g56/te9VCiFICuCHUXs1kNBbiqsif8kf YYIgMKMBBEchQ9GPAk1CrBVp7b7Rigs+ byao4IYT21Tqgale9RCUep

In [2]:
a_query = dns.message.make_query('paypal.com', 'A', want_dnssec=True)
response1 = dns.query.udp(a_query, '198.41.0.4')
print(response1)
print("***"*5)
ds = None
rrsig = None
for i in response1.authority:
    print(i.rdtype)
    if not ds and i.rdtype == 43:
        ds = i
    elif not rrsig and i.rdtype == 46:
        rrsig = i
    if ds and rrsig:
        break
print("DS: ", ds)
print("RRSIG: ", rrsig)    

id 51243
opcode QUERY
rcode NOERROR
flags QR RD
edns 0
eflags DO
payload 1472
;QUESTION
paypal.com. IN A
;ANSWER
;AUTHORITY
com. 172800 IN NS a.gtld-servers.net.
com. 172800 IN NS b.gtld-servers.net.
com. 172800 IN NS c.gtld-servers.net.
com. 172800 IN NS d.gtld-servers.net.
com. 172800 IN NS e.gtld-servers.net.
com. 172800 IN NS f.gtld-servers.net.
com. 172800 IN NS g.gtld-servers.net.
com. 172800 IN NS h.gtld-servers.net.
com. 172800 IN NS i.gtld-servers.net.
com. 172800 IN NS j.gtld-servers.net.
com. 172800 IN NS k.gtld-servers.net.
com. 172800 IN NS l.gtld-servers.net.
com. 172800 IN NS m.gtld-servers.net.
com. 86400 IN DS 30909 8 2 e2d3c916f6deeac73294e8268fb5885044a833fc5459588f4a9184cfc41a5766
com. 86400 IN RRSIG DS 8 1 86400 20211005210000 20210922200000 26838 . bF/r3zrgxQLjC3hituZPxmzTURcKJ3nc 8BjFu0eMJwxovRYJBQwLetYcWL0c5HEl GkzGA/wWZJ1XjxRaCpIaw040WbVwX7+T CKR3nrNS8XbKWb/u5NRZERafRKxw+BbU g56/te9VCiFICuCHUXs1kNBbiqsif8kf YYIgMKMBBEchQ9GPAk1CrBVp7b7Rigs+ byao4IYT21Tqgale9RCUe

In [3]:
ds[0]
type(rrsig[0])

dns.rdtypes.ANY.RRSIG.RRSIG

In [4]:
b_query = dns.message.make_query('.', 'DNSKEY', want_dnssec=True)
response2 = dns.query.udp(b_query, '198.41.0.4')
print(response2)

id 53504
opcode QUERY
rcode NOERROR
flags QR AA RD
edns 0
eflags DO
payload 1472
;QUESTION
. IN DNSKEY
;ANSWER
. 172800 IN DNSKEY 256 3 8 AwEAAY+oUaY0b7Z45vRD1ef/GykZqgHJ tfdzRcnQNvGVQAqlH22QChtG+n1EMugw 7T/6uDBAGlRIkXASdtHXhxStb9lPpyQe 5/JIuMIlg+NhxKxEJ5e3J9SSPCavvDhH /BPrBCJwn8b68QAWRjVW6Rgdx63pUm7l fsimiWGMfplHNvcZWgVbKA9OI2o2lU8r T8n7zuwtlZPNpDLSI5GzrJgIiKR2Id16 fmAgTJBOw14Xye/t4/BxTdxeMiiVFwA4 KUV2VeqspHKSHFOz+lUIIqBRknEmYpSv nxnyi0n1n4tGnGP8z6ZwRACi1Rw0nCu7 BGOU9M6LpInRoW/W4KXLODr6xqU=
. 172800 IN DNSKEY 256 3 8 AwEAAbDEyqdwu2fqAwinPCFwALUCWfYY aLrNhnOrMxDorLBYMipEE1btlK1XnigT RMeb0YQ8/LCopb3CN73hYDhCHFsNk+Gt ukBB+gWLcg+2FZXbhLXIheQm8x2VfOHy 2yYQG+18wjx3HY9Mj/ZEhXbZNrDMvpFK KVihWXa0/cHNg4ZcIHD9KkMlKzK+my1K /vz8fq5cFCFOu7wgM+kKbOikdcRBm7Uf /wRXZItFg2uhUijUb56gEN8uCUgmuEw6 wQ5ZBuR7UT/FLyyAUeAH87oxF4im2DXK 6J+JA7IAs2UHJ16uTqvdserUU8NIosis laXIZCvz+NTDb3SJcxs6bvCikeU=
. 172800 IN DNSKEY 257 3 8 AwEAAaz/tAm8yTn4Mfeh5eyI96WSVexT BAvkMgJzkKTOiW1vkIbzxeF3+/4RgWOq 7HrxRixHlFlExOLAJr5emLvN

In [5]:
response2.answer
type(response2.answer[0])

response2Dnskey = response2.answer[0]
response2Rrsig = response2.answer[1]

print(dns.dnssec.validate(response2Dnskey, response2Rrsig, {key : response2Dnskey}))

None


In [6]:
root_hash=["49AAC11D7B6F6446702E54A1607371607A1A41855200FD2CE1CDDE32F24E8FB5".lower(),"E06D44B80B8F1D39A95C0B0D7C65D08458E880409BBC683457104237C7F8EC8D".lower()]

In [7]:
for res in response2Dnskey:
    print(res.rdtype)
    if "257" in str(res):
        zsk = res
        break
zskDecrypt = dns.dnssec.make_ds(".", zsk, "SHA256", origin=None)
zskDecrypt = str(zskDecrypt)
print(zskDecrypt.split(" ")[3])
zskDecrypt = (zskDecrypt.split(" ")[3])

RdataType.DNSKEY
RdataType.DNSKEY
RdataType.DNSKEY
e06d44b80b8f1d39a95c0b0d7c65d08458e880409bbc683457104237c7f8ec8d


In [8]:
if str(zskDecrypt) in root_hash:
    print("All good, munduku po")
else:
    print("Nope, zsk corrupted")

All good, munduku po


In [9]:
print(dns.dnssec.validate(response2.answer[0], response2.answer[1], {key:response2.answer[0]}))

None


In [10]:
response2Rrsig[0]

<DNS IN RRSIG(DNSKEY) rdata: DNSKEY 8 0 172800 20211010000000 20210919000000 20326 . Sr5anUGJStj+Ve2zr83JMQrEmbkf88Ri 52nFderay/sSIad8vWAxiPoxdaJ1iWky hJSWeOR0Sw4lRsphPIj54s3yqpPfXUg8 rPoIVeYYYkTFO5W5BF3gLsMfu2ZHaHLp NOsy6GCgPwXCxq49lJ/xcLyS0tjGkSH8 IW3sAT15sXDhLkMjMISJQ776Z2LSonXc NvNeGrkXuzCmcOwdduWnaSFsmKoPWQbb 3a4kANk0I0GRtDIp8XApb6WnNAmKTPRa H/VoX9M6b8CZyp5TuAbNPnW8nVFaCRMp nWCWYYOUhLm7jYEyNSr+V+/ygD+OUyrx 4R/WzIGSHaiHGQB1KU4eyw==>

In [11]:
print(dns.name.from_text("google.com."))

google.com.


In [12]:
key=dns.name.from_text(".")
value = dns.dnssec.validate(response2.answer[0], response2.answer[1], {key:response2.answer[0]})
print(value)

None


In [13]:
response1.authority

[<DNS com. IN NS RRset: [<a.gtld-servers.net.>, <b.gtld-servers.net.>, <c.gtld-servers.net.>, <d.gtld-servers.net.>, <e.gtld-servers.net.>, <f.gtld-servers.net.>, <g.gtld-servers.net.>, <h.gtld-servers.net.>, <i.gtld-servers.net.>, <j.gtld-servers.net.>, <k.gtld-servers.net.>, <l.gtld-servers.net.>, <m.gtld-servers.net.>]>,
 <DNS com. IN DS RRset: [<30909 8 2 e2d3c916f6deeac73294e8268fb5885044a833fc5459588f4a9184cfc41a5766>]>,
 <DNS com. IN RRSIG(DS) RRset: [<DS 8 1 86400 20211005210000 20210922200000 26838 . bF/r3zrgxQLjC3hituZPxmzTURcKJ3nc 8BjFu0eMJwxovRYJ...>]>]

In [14]:
print(dns.dnssec.make_ds(".", response2.answer[0][0], "SHA256", origin=None))
key=dns.name.from_text(".")
print(type(key))
print(response1.authority[0])
# print(response1.authority[2])
value = dns.dnssec.validate(response2.answer[0], response2.answer[1], {key:response2.answer[0]})
value = dns.dnssec.validate(response1.authority[1], response1.authority[2], {key:response2.answer[0]})

14748 8 2 6d698396855c5606482d3785e8fb0cb94259adade6308d3ad67d228d09fd9aa2
<class 'dns.name.Name'>
com. 172800 IN NS a.gtld-servers.net.
com. 172800 IN NS b.gtld-servers.net.
com. 172800 IN NS c.gtld-servers.net.
com. 172800 IN NS d.gtld-servers.net.
com. 172800 IN NS e.gtld-servers.net.
com. 172800 IN NS f.gtld-servers.net.
com. 172800 IN NS g.gtld-servers.net.
com. 172800 IN NS h.gtld-servers.net.
com. 172800 IN NS i.gtld-servers.net.
com. 172800 IN NS j.gtld-servers.net.
com. 172800 IN NS k.gtld-servers.net.
com. 172800 IN NS l.gtld-servers.net.
com. 172800 IN NS m.gtld-servers.net.


In [15]:
b_query = dns.message.make_query('.', 'DNSKEY', want_dnssec=True)
response2 = dns.query.udp(b_query, '198.41.0.4')
print(response2)

id 24391
opcode QUERY
rcode NOERROR
flags QR AA RD
edns 0
eflags DO
payload 4096
;QUESTION
. IN DNSKEY
;ANSWER
. 172800 IN DNSKEY 256 3 8 AwEAAY+oUaY0b7Z45vRD1ef/GykZqgHJ tfdzRcnQNvGVQAqlH22QChtG+n1EMugw 7T/6uDBAGlRIkXASdtHXhxStb9lPpyQe 5/JIuMIlg+NhxKxEJ5e3J9SSPCavvDhH /BPrBCJwn8b68QAWRjVW6Rgdx63pUm7l fsimiWGMfplHNvcZWgVbKA9OI2o2lU8r T8n7zuwtlZPNpDLSI5GzrJgIiKR2Id16 fmAgTJBOw14Xye/t4/BxTdxeMiiVFwA4 KUV2VeqspHKSHFOz+lUIIqBRknEmYpSv nxnyi0n1n4tGnGP8z6ZwRACi1Rw0nCu7 BGOU9M6LpInRoW/W4KXLODr6xqU=
. 172800 IN DNSKEY 257 3 8 AwEAAaz/tAm8yTn4Mfeh5eyI96WSVexT BAvkMgJzkKTOiW1vkIbzxeF3+/4RgWOq 7HrxRixHlFlExOLAJr5emLvN7SWXgnLh 4+B5xQlNVz8Og8kvArMtNROxVQuCaSnI DdD5LKyWbRd2n9WGe2R8PzgCmr3EgVLr jyBxWezF0jLHwVN8efS3rCj/EWgvIWgb 9tarpVUDK/b58Da+sqqls3eNbuv7pr+e oZG+SrDK6nWeL3c6H5Apxz7LjVc1uTId sIXxuOLYA4/ilBmSVIzuDWfdRUfhHdY6 +cn8HFRm+2hM8AnXGXws9555KrUB5qih ylGa8subX2Nn6UwNR1AkUTV74bU=
. 172800 IN DNSKEY 256 3 8 AwEAAbDEyqdwu2fqAwinPCFwALUCWfYY aLrNhnOrMxDorLBYMipEE1btlK1XnigT RMeb0YQ8/LCopb3CN73hYDhC

In [84]:
def validateDnsSec(response1, response2, qtext):
#     noDs = True
#     for i in response1.additional:
#         if i.rdtype == 43:
#             noDs = False
#     if noDs:
#         print("DNSSEC NOT SUPPORTED")
#         sys.exit(0)
    ds = None
    rrsig = None
    if response1.answer:
        print("***"*10)
        print("***"*10)
        print("***"*10)
        print("***"*10)
        print("ANSWER HERE")
        print("***"*10)
        print("***"*10)
        print("***"*10)
#         print(response1.answer[1][0])
        return True
#     for i in response1.authority:
# #         print(i.rdtype)
#         if not ds and i.rdtype == 43:
#             ds = i
#         elif not rrsig and i.rdtype == 46:
#             rrsig = i
#         if ds and rrsig:
#             break
#     if not ds or not rrsig:
#         print("DS:",ds)
#         print("rrsig:",rrsig)
#         print(response1)
#         raise Exception
#     print(response2)
    response2Dnskey = response2.answer[0]
#     response2Rrsig = response2.answer[1]
    for res in response2Dnskey:
#     print(res.rdtype)
        if "257" in str(res):
            zsk = res
            break
    zskDecrypt = dns.dnssec.make_ds(qtext, zsk, "SHA256", origin=None)
    zskDecrypt = str(zskDecrypt)
#     print("ZSK:", zskDecrypt)
#     print(zskDecrypt.split(" ")[3])
    zskDecrypt = (zskDecrypt.split(" ")[3])
    if not str(zskDecrypt) in root_hash:
        print("ZSK not in hash!!".upper())
        raise Exception
        
    key=dns.name.from_text(qtext)
    try:
        if not dns.dnssec.validate(response2.answer[0], response2.answer[1], {key:response2.answer[0]}):
            pass
        else:
            return False
    except Exception as e:
        print("Validation Failed!")
        if not dns.dnssec.validate(response1.authority[1], response1.authority[2], {key:response2.answer[0]}):
            pass
        else:
            return False
    except Exception as e:
        return False
#     print("DNSSEC VALIDATED at \"{}\"".format(qtext))
    return True

In [75]:
a_query = dns.message.make_query('paypal.com', 'A', want_dnssec=True)
response1 = dns.query.udp(a_query, '198.41.0.4')
b_query = dns.message.make_query('.', 'DNSKEY', want_dnssec=True)
response2 = dns.query.udp(b_query, '198.41.0.4')
validateDnsSec(response1, response2, ".")

True

In [76]:
import sys

In [77]:
def fetchResponseWithValidation(url, dnsDomain, rtype, root):
#     print("Root check: ", root)
    response = fetchResponse(url, rtype, root)
    responseDns = fetchResponse(dnsDomain, "DNSKEY", root)
#     print("dnsDomain: ")
    responseForValidation = validateDnsSec(response, responseDns, dnsDomain)
#     print("Validation: ", responseForValidation)
    if not responseForValidation:
        print("DNSSEC FAILED")
        return response, False
                
    else:
        print("DNS OKAY")
        return response, True

In [89]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Sep 19 16:02:28 2021

@author: vishnutammishetti
"""

import dns.resolver
import time

#Funcs
def fetchResponse(queryText, rtype, targetIp):
#     print("In fetch for ", queryText, "type ", rtype)
    query = dns.message.make_query(queryText, rtype, want_dnssec=True)  
    response = dns.query.udp(query, targetIp, timeout = 2)
    return response

def ipsIn(result):
    for i in result:
        if i.rdtype == 1:
            return True
    return False
def nssIn(result):
    for i in result:
        if i.rdtype == 2:
            return True
    return False
def mxsIn(result):
    for i in result:
        if i.rdtype == 15:
            return True
    return False

def printResult(url, rType, resultSet, timeTaken):
    print("QUESTION SECTION:")
    print("{}     IN      {}".format(url, rType))
    
    print("ANSWER SECTION:")
    print(resultSet[0])
    print("")
    print("Query time: {} msec".format(int(timeTaken)))
    print("When: {}".format(time.ctime()))

def mainFunction(url, rType, dnsDomain = ".", dnsCounter = 0):
    global root_hash
#     print("URL: ", url)
    for root in root_servers.values():
        if True:
            response, responseDns = fetchResponseWithValidation(url, dnsDomain, rType, root)
#             for i in response.authority:
#                 print("Auth:", i)
            if not response.answer:
                root_hash.append(response.authority[1].to_text().split(" ")[7])
#             response = fetchResponse(url, rType, root)
#             response, responseDns = fetchResponseWithValidation(url, dnsDomain, rType, root)
#             responseDns = fetchResponse(dnsDomain, "DNSKEY", root)
#             responseForValidation = validateDnsSec(response, responseDns, dnsDomain)
#             print("Validation: ", responseForValidation)
#             if not responseForValidation:
#                 print("DNSSEC FAILED")
                
#             else:
#                 print("DNS OKAY")
#                 root_hash.append(response1.authority[1].to_text().split(" ")[7])
#                 print("ROOT HASH:", root_hash)
            if responseDns:
                dnsCounter += 1
                urlList = url.split(".")
                dnsDomain = ".".join(urlList[-dnsCounter:]) + "."
            else:
                print("DNS Failed")
                return response, False
#                 sys.exit(0)
#                     print("DNS DOMAIN: ",dnsDomain)
#             print("Hello")
#             print(response)
            while not response.answer:
#                 print(response)
#                 print("IN NOT ANSWER")
                if response.additional:
                    for val in response.additional:
    #                     print(val)
                        if val.rdtype == 1:
                            nextIp = str(val[0])
                            response, responseDns = fetchResponseWithValidation(url, dnsDomain, rType, nextIp)
                            
                            if not response.answer:
                                root_hash.append(response.authority[1].to_text().split(" ")[7])
                            
                            if not responseDns:
                                print("DNS Failed")
                                return response, False
                            if responseDns:
                                dnsCounter += 1
                                urlList = url.split(".")
                                dnsDomain = ".".join(urlList[-dnsCounter:]) + "."
#                             response = fetchResponse(url, rType, nextIp)
                            
                            break

                else:
                    if response.authority:
                        for val in response.authority:
                            if val.rdtype == 2:
                                nameServer = str(val[0])
                                break
                        nameServerResponse = mainFunction(nameServer, "A", dnsDomain, dnsCounter)
#                         print(nameServerResponse)
    #                     return
                        for i in nameServerResponse.answer:
                            foundNs = False
                            if i.rdtype == 1:
                                nextIp = i
#                                 print(nextIp)
                                response.additional = [nextIp]
                                foundNs = True
                                break
                        if not foundNs:
                            return response, True # Not sure!
                        # If there is no name server as well, the authority section might contain the NS. Check dig www.google.com NS
            if nssIn(response.answer) and rType == "NS":
                return response, True
            if mxsIn(response.answer) and rType == "MX":
                return response, True
            if ipsIn(response.answer):
                return response, True
            else:
                print("NO IPS")
                cname = None
                for i in response.answer:
                    if i.rdtype == 5:
                        print("Has CNAMES")
                        cname = str(i)
                        
                        break
                if cname:
#                     print("In loop")
                    response1 = mainFunction(str(i), rType)
#                     print("Called Main function again!")
#                     print(response1)
                    return ((response1), True)
                else:
                    return (response, True)
#                     break
#                 return response
#         except Exception as e:
# #             raise e
#             print(e)
#             continue
        
def finalFunc(url, rtype):
    startTime = time.time()
    finalUrl = url[::]
    if url[:4] == "www.":
        finalUrl = finalUrl[4::]
    response, dns = mainFunction(finalUrl, rtype)
    timeTaken = (time.time() - startTime)*10**3
    if dns:
        
        print("Final response:\n")
#     print(response.answer)
        
        printResult(url, rtype, response.answer, timeTaken)
    else:
        print("DNSSEC VALIDATION FAILED")
    
    return timeTaken
    

#Funcs

root_servers = {}

root_servers['a'] = '198.41.0.4'
root_servers['b'] = '199.9.14.201'
root_servers['c'] = '192.33.4.12'
root_servers['d'] = '199.7.91.13'
root_servers['e'] = '192.203.230.10'
root_servers['f'] = '192.5.5.241'
root_servers['g'] = '192.112.36.4'
root_servers['h'] = '198.97.190.53'
root_servers['i'] = '192.36.148.17'
root_servers['j'] = '192.58.128.30'
root_servers['k'] = '193.0.14.129'
root_servers['l'] = '199.7.83.42'
root_servers['n'] = '202.12.27.33'


inputString = input()
inputList = inputString.split()
if inputList[0] != "mydig":
    print("ERROR, Please enter mydig website rtype")
else:
    
    finalFunc(inputList[1], inputList[2])
# mydig cs.stonybrook.edu A
# mydig google.co.jp A
# mydig www.dnssec-fail.org A

mydig paypal.com A
DNS OKAY
DNS OKAY
******************************
******************************
******************************
******************************
ANSWER HERE
******************************
******************************
******************************
DNS OKAY
Final response:

QUESTION SECTION:
paypal.com     IN      A
ANSWER SECTION:
paypal.com. 300 IN A 64.4.250.37
paypal.com. 300 IN A 64.4.250.36

Query time: 165 msec
When: Wed Sep 22 19:40:02 2021


In [270]:
url = "cs.google.com"
dnsDomain = "."
dnsCounter = 0

In [167]:
dnsCounter += 1
urlList = url.split(".")
dnsDomain = ".".join(urlList[-dnsCounter:]) + "."

In [168]:
dnsDomain

'cs.google.com.'

In [170]:
print(root_hash)

['49aac11d7b6f6446702e54a1607371607a1a41855200fd2ce1cdde32f24e8fb5', 'e06d44b80b8f1d39a95c0b0d7c65d08458e880409bbc683457104237c7f8ec8d']


In [243]:
print(response1)

id 20932
opcode QUERY
rcode NOERROR
flags QR RD
edns 0
eflags DO
payload 1472
;QUESTION
paypal.com. IN A
;ANSWER
;AUTHORITY
com. 172800 IN NS a.gtld-servers.net.
com. 172800 IN NS b.gtld-servers.net.
com. 172800 IN NS c.gtld-servers.net.
com. 172800 IN NS d.gtld-servers.net.
com. 172800 IN NS e.gtld-servers.net.
com. 172800 IN NS f.gtld-servers.net.
com. 172800 IN NS g.gtld-servers.net.
com. 172800 IN NS h.gtld-servers.net.
com. 172800 IN NS i.gtld-servers.net.
com. 172800 IN NS j.gtld-servers.net.
com. 172800 IN NS k.gtld-servers.net.
com. 172800 IN NS l.gtld-servers.net.
com. 172800 IN NS m.gtld-servers.net.
com. 86400 IN DS 30909 8 2 e2d3c916f6deeac73294e8268fb5885044a833fc5459588f4a9184cfc41a5766
com. 86400 IN RRSIG DS 8 1 86400 20211005210000 20210922200000 26838 . bF/r3zrgxQLjC3hituZPxmzTURcKJ3nc 8BjFu0eMJwxovRYJBQwLetYcWL0c5HEl GkzGA/wWZJ1XjxRaCpIaw040WbVwX7+T CKR3nrNS8XbKWb/u5NRZERafRKxw+BbU g56/te9VCiFICuCHUXs1kNBbiqsif8kf YYIgMKMBBEchQ9GPAk1CrBVp7b7Rigs+ byao4IYT21Tqgale9RCUe